In [4]:
%pip install pandas
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.6 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 811.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/45.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/45.9 MB 14.1 MB/s eta 0:00:04
    --------------------------------------- 0.6/45.9 MB 10.2 MB/s eta 0:00:05
    --------------------------------------- 1.0/45.9 MB 9.5 MB/s eta 0:00:05
   - -------------------------------------- 1.5/45.9 MB 9.8 MB/s eta 0:00:05
   - -------------------------------------- 2.1/45.9 MB 10.1 MB/s eta 0:00:05
   -- ------------------------------------- 2.6/45.9 MB 10.4 MB/s eta 0:00:05
   -- ------------------------------------- 3.1/45.9 MB 10.5 MB/s eta 0:00:05
   --- ------------------------------------ 3.7/45.9 MB 10.6 MB/s eta 0:00:04
   --- ---

In [53]:
import pandas as pd
import json
import numpy as np
from scipy import stats
from datetime import datetime

In [49]:
year = 2017
inmet_weather = 0
for i in range(6):
    year+=1
    print(year)
    if i == 0:
        inmet_weather = pd.read_csv(f"../INMET_weather/weather_{year}/weather_{year}.csv", on_bad_lines='skip')
    else:
        inmet_weather = pd.concat([inmet_weather, pd.read_csv(f"../INMET_weather/weather_{year}/weather_{year}.csv", sep=",")], ignore_index=True, sort=False)

2018
2019
2020
2021
2022
2023


In [50]:
inmet_weather.drop(columns=["RADIACAO GLOBAL (KJ/m²)"],inplace=True)
inmet_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30487128 entries, 0 to 30487127
Data columns (total 19 columns):
 #   Column                                                 Dtype  
---  ------                                                 -----  
 0   DATA (YYYY-MM-DD)                                      object 
 1   Hora UTC                                               object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       float64
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  float64
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        float64
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       float64
 6   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           float64
 7   TEMPERATURA DO PONTO DE ORVALHO (°C)                   float64
 8   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)             float64
 9   TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)             float64
 10  TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)       float64
 

In [59]:
inmet_train = inmet_weather[inmet_weather["DATA (YYYY-MM-DD)"].apply(lambda x: int(x[:4]))<=2020]
inmet_val = inmet_weather[inmet_weather["DATA (YYYY-MM-DD)"].apply(lambda x: int(x[:4]))>2020]
inmet_val = inmet_val[inmet_val["DATA (YYYY-MM-DD)"].apply(lambda x: int(x[:4]))<=2022]
inmet_test = inmet_weather[inmet_weather["DATA (YYYY-MM-DD)"].apply(lambda x: int(x[:4]))>2022]


In [60]:
inmet_train.groupby(inmet_train["DATA (YYYY-MM-DD)"].apply(lambda x: x[:4])).size()

DATA (YYYY-MM-DD)
2018    5128416
2019    5118216
2020    5173776
dtype: int64

In [61]:
inmet_val.groupby(inmet_val["DATA (YYYY-MM-DD)"].apply(lambda x: x[:4])).size()

DATA (YYYY-MM-DD)
2021    5150880
2022    4948920
dtype: int64

In [62]:
inmet_test.groupby(inmet_test["DATA (YYYY-MM-DD)"].apply(lambda x: x[:4])).size()

DATA (YYYY-MM-DD)
2023    4966920
dtype: int64

In [45]:
inmet_weather.dropna(inplace=True)
inmet_weather.reset_index(drop=True, inplace=True)
inmet_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19922658 entries, 0 to 19922657
Data columns (total 19 columns):
 #   Column                                                 Dtype  
---  ------                                                 -----  
 0   DATA (YYYY-MM-DD)                                      object 
 1   Hora UTC                                               object 
 2   PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                       float64
 3   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  float64
 4   PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)        float64
 5   PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)       float64
 6   TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)           float64
 7   TEMPERATURA DO PONTO DE ORVALHO (°C)                   float64
 8   TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)             float64
 9   TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)             float64
 10  TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)       float64
 

In [63]:
inmet_train.isna().sum()

DATA (YYYY-MM-DD)                                              0
Hora UTC                                                       0
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                         2966580
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    2191269
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          2202586
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         2202683
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             2174463
TEMPERATURA DO PONTO DE ORVALHO (°C)                     2637654
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               2186116
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               2186338
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         2652289
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         2661927
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                 2640834
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                 2656452
UMIDADE RELATIVA DO AR, HORARIA (%)                      2637936
VENTO, DIREÇÃO HORARIA (g

In [66]:
inmet_val.isna().sum()

DATA (YYYY-MM-DD)                                              0
Hora UTC                                                       0
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                         4343501
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    3737388
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          3759366
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         3759362
TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             3752796
TEMPERATURA DO PONTO DE ORVALHO (°C)                     4069588
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               3777591
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               3775152
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         4094247
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         4094419
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                 4076860
UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)                 4074313
UMIDADE RELATIVA DO AR, HORARIA (%)                      4051707
VENTO, DIREÇÃO HORARIA (g

In [65]:
inmet_val

,DATA (YYYY-MM-DD),Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",ESTACAO
15420408,2021-01-01,0000 UTC,0.0,887.1,887.1,886.3,20.9,15.9,22.4,20.8,16.5,15.4,75.0,67.0,73.0,327.0,3.6,0.8,A001
15420409,2021-01-01,0100 UTC,0.0,887.3,887.5,887.1,20.5,16.4,21.1,19.5,16.9,16.0,83.0,73.0,78.0,134.0,2.3,1.5,A001
15420410,2021-01-01,0200 UTC,0.0,887.9,888.0,887.2,19.6,16.1,21.4,19.6,16.5,15.9,80.0,73.0,80.0,309.0,2.5,1.1,A001
15420411,2021-01-01,0300 UTC,0.0,887.6,888.0,887.6,19.4,16.7,20.1,19.3,17.0,16.1,86.0,78.0,84.0,315.0,2.3,0.6,A001
15420412,2021-01-01,0400 UTC,0.0,887.0,887.6,887.0,18.7,16.3,19.4,18.6,16.8,16.0,86.0,83.0,86.0,270.0,1.4,0.7,A001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25520203,2022-12-31,1900 UTC,0.0,906.2,906.9,906.2,28.5,13.6,29.1,27.6,15.2,13.3,46.0,39.0,40.0,173.0,6.0,1.9,A898
25520204,2022-12-31,2000 UTC,0.0,905.9,906.3,905.9,26.9,13.6,29.1,26.8,14.9,13.4,45.0,40.0,44.0,95.0,5.8,2.8,A898
25520205,2022-12-31,2100 UTC,0.0,905.8,906.0,905.7,26.6,15.4,26.9,25.4,15.8,13.5,55.0,44.0,50.0,35.0,8.0,3.9,A898
25520206,2022-12-31,2200 UTC,0.0,905.8,905.8,905.7,24.7,15.6,26.7,24.7,15.6,14.2,57.0,48.0,57.0,56.0,6.9,3.8,A898
